In [13]:
from cust_functions.explain_helper import *
import pandas as pd 


In [14]:
# ResGCN on AKI data
Features_AKI_ResGCN = create_avg_sd_df('./explanations/AKI_ResGCN.pkl', True, True)
Pathways_AKI_ResGCN = create_avg_sd_df('./explanations/AKI_ResGCN.pkl', False, True)
Features_Pert_AKI_ResGCN = create_avg_sd_df('./explanations/Pert_AKI_ResGCN.pkl', True, True)
Pathways_Pert_AKI_ResGCN = create_avg_sd_df('./explanations/Pert_AKI_ResGCN.pkl', False, True)

# ResGCN on Covid data 
Features_Covid_ResGCN = create_avg_sd_df('./explanations/Covid_ResGCN.pkl', True, True)
Pathways_Covid_ResGCN = create_avg_sd_df('./explanations/Covid_ResGCN.pkl', False, True)
Features_Pert_Covid_ResGCN = create_avg_sd_df('./explanations/Pert_Covid_ResGCN.pkl', True, True)
Pathways_Pert_Covid_ResGCN = create_avg_sd_df('./explanations/Pert_Covid_ResGCN.pkl', False, True)

# ResGAT on AKI data
Features_AKI_ResGAT = create_avg_sd_df('./explanations/AKI_ResGAT.pkl', True, True)
Pathways_AKI_ResGAT = create_avg_sd_df('./explanations/AKI_ResGAT.pkl', False, True)

# ResGAT on Covid data
Features_AKI_ResGAT = create_avg_sd_df('./explanations/Covid_ResGAT.pkl', True, True)
Pathways_AKI_ResGAT = create_avg_sd_df('./explanations/Covid_ResGAT.pkl', False, True)


In [16]:
Features_AKI_ResGCN.head(20)

,Protein,Avg_Feature_Score,Sd_Feature_score
62,P02751,1.867254,1.010641
93,P02679,1.686572,0.910365
52,P01834,1.630119,0.870249
373,P01763,1.619361,0.815459
433,P01764,1.597320,0.814249
1,P02671,1.595159,0.878772
485,P01717,1.550195,0.828298
387,P01742,1.518247,0.777355
224,P06312,1.511713,0.817365
447,P01593,1.508734,0.780777
